In [17]:
from rdkit import Chem
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import MolsToGridImage
import pandas as pd

In [18]:
from rdkit import rdBase
blocker = rdBase.BlockLogs()

### Read in test data

In [19]:
test_data = pd.read_csv("test_polymerization.csv", index_col=None)

#### Do polymerizations

In [20]:
from polymerize2 import get_polymer_list

In [21]:
mol_lists = []

for _, row in test_data.iterrows():
    try:
        mol_list, SMILES = get_polymer_list(degree_of_polymerization=3, monomer_SMILES=row['Canonical SMILES'], polymerization_type=row['BASE_Category'], end_group="C")
        mol_lists.append(mol_list)
    except:
        print(f"{row["Canonical SMILES"]},{row['BASE_Category']}")
        mol_lists.append([])
test_data['Mols'] = mol_lists

C[Si]1(C)O[Si](C)(C)O[Si](C)(C)O[Si](C)(C)O1,cationic
C[Si]1(C)O[Si](C)(C)O[Si](C)(C)O1,cationic
C1OCOCO1,cationic
C1OCOCO1,cationic
C1OCOCO1,cationic
CC[Si]1(CC)O[Si](CC)(CC)O[Si](CC)(CC)O1,cationic
O=C1CSSCC(=O)O1,cationic
O=C1CSSCC(=O)O1,cationic
O=C1CSSCC(=O)O1,cationic
C1OCOCO1,cationic
C1OCOCO1,cationic
CN1[Si](C)(C)O[Si](C)(C)O[Si]1(C)C,cyclic


### Generate PDF

In [22]:
from rdkit import Chem
from rdkit.Chem import Draw
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io

In [23]:
def draw_molecules_to_pdf(mol_lists, output_filename="molecules.pdf"):
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter
    
    for page_num, mol_list in enumerate(mol_lists, start=1):
        c.setFont("Helvetica", 10)
        
        # Define positions for the molecules
        positions = [(50, 500), (300, 500), (50, 250), (300, 250)]
        
        for i, mol in enumerate(mol_list):
            if mol:
                smiles = Chem.MolToSmiles(mol)
                img = Draw.MolToImage(mol, size=(200, 200))
                img_io = io.BytesIO()
                img.save(img_io, format='PNG')
                img_io.seek(0)
                
                x, y = positions[i]
                c.drawImage(ImageReader(img_io), x, y, width=200, height=200)

            if i == 0:
                x, y = positions[i]
                c.drawString(x, y + 210, smiles)  # Draw SMILES above image
        
        # Add page number at the bottom
        c.drawString(width / 2 - 20, 30, f"Page {page_num}")
        c.showPage()
    
    c.save()
    print(f"PDF saved as {output_filename}")

In [ ]:
draw_molecules_to_pdf(mol_lists)